In [1]:
# first run installations and data download
# ! llama-index-llms-openai llama-index-readers-file llama-index llama-index-llms-huggingface llama-index-vector-stores-milvus pymilvus
# ! mkdir -p './data/10k/'
# ! curl 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/examples/data/10k/uber_2021.pdf' -O 'data/10k/uber_2021.pdf'
# ! curl 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/examples/data/10k/lyft_2021.pdf' -O 'data/10k/lyft_2021.pdf'
# ! mv 'lyft_2021.pdf' 'uber_2021.pdf' './data/10k/'

In [1]:
from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    StorageContext,
    load_index_from_storage,
)
from llama_index.core.tools import QueryEngineTool, ToolMetadata

from pymilvus import connections
from llama_index.vector_stores.milvus import MilvusVectorStore

In [2]:
# try:
#     storage_context = StorageContext.from_defaults(
#         persist_dir="./storage/lyft"
#     )
#     lyft_index = load_index_from_storage(storage_context)

#     storage_context = StorageContext.from_defaults(
#         persist_dir="./storage/uber"
#     )
#     uber_index = load_index_from_storage(storage_context)

#     index_loaded = True
# except:
    # index_loaded = False
index_loaded=False

In [3]:
if not index_loaded:
    # load data
    lyft_docs = SimpleDirectoryReader(
        input_files=["./data/10k/lyft_2021.pdf"]
    ).load_data()
    uber_docs = SimpleDirectoryReader(
        input_files=["./data/10k/uber_2021.pdf"]
    ).load_data()

    # build index
    vector_store_lyft = MilvusVectorStore(dim=1536, collection_name="lyft", overwrite=True)
    vector_store_uber = MilvusVectorStore(dim=1536, collection_name="uber", overwrite=True)
    storage_context_lyft = StorageContext.from_defaults(vector_store=vector_store_lyft)
    storage_context_uber = StorageContext.from_defaults(vector_store=vector_store_uber)
    lyft_index = VectorStoreIndex.from_documents(lyft_docs, storage_context=storage_context_lyft)
    uber_index = VectorStoreIndex.from_documents(uber_docs, storage_context=storage_context_uber)

    # persist index
    lyft_index.storage_context.persist(persist_dir="./storage/lyft")
    uber_index.storage_context.persist(persist_dir="./storage/uber")

In [4]:
lyft_engine = lyft_index.as_query_engine(similarity_top_k=3)
uber_engine = uber_index.as_query_engine(similarity_top_k=3)

In [5]:
query_engine_tools = [
    QueryEngineTool(
        query_engine=lyft_engine,
        metadata=ToolMetadata(
            name="lyft_10k",
            description=(
                "Provides information about Lyft financials for year 2021. "
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    ),
    QueryEngineTool(
        query_engine=uber_engine,
        metadata=ToolMetadata(
            name="uber_10k",
            description=(
                "Provides information about Uber financials for year 2021. "
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    ),
]

In [6]:
from llama_index.core.agent import ReActAgent
from llama_index.llms.huggingface import HuggingFaceLLM

/Users/yujiantang/Documents/workspace/ai_agents_cookbooks/newenv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
llm = HuggingFaceLLM(
    context_window=2048,
    max_new_tokens=256,
    generate_kwargs={"temperature": 0.25, "do_sample": False},
    # query_wrapper_prompt=query_wrapper_prompt,
    tokenizer_name="llmware/bling-sheared-llama-2.7b-0.1",
    model_name="llmware/bling-sheared-llama-2.7b-0.1",
    device_map="cpu",
    tokenizer_kwargs={"max_length": 2048},
    # uncomment this if using CUDA to reduce memory usage
    # model_kwargs={"torch_dtype": torch.float16}
)

agent = ReActAgent.from_tools(
    query_engine_tools,
    llm=llm,
    verbose=True,
    # context=context
)

In [11]:
response = agent.chat("What was Lyft's revenue growth in 2021?")
print(str(response))

KeyboardInterrupt: 